## This Lab is an example of how to work with data

### Steps
- 1: Read Data
- 2: Clean/Prep Data
- 3: Summarize Data

#### Step 1: Read from Parquet File

In [4]:
parquetDF = spark.read.parquet("/tmp/databricks-df-example.parquet")
display(parquetDF)

#### Step 2: Clean/Prep Data

In [6]:
#Explode the employees column
from pyspark.sql.functions import explode

explodeDF = unionDF.select(explode("employees").alias("e"))
flattenDF = explodeDF.selectExpr("e.firstName", "e.lastName", "e.email", "e.salary")

flattenDF.show()

In [7]:
#Simple filtering
filterDF = flattenDF.filter(flattenDF.firstName == "xiangrui").sort(flattenDF.lastName)
display(filterDF)

In [8]:
#More filtering
from pyspark.sql.functions import col, asc

# Use `|` instead of `or`
filterDF = flattenDF.filter((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
display(filterDF)

In [9]:
#More filtering
whereDF = flattenDF.where((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
display(whereDF)

In [10]:
#Replace the null values
nonNullDF = flattenDF.fillna("--")
display(nonNullDF)

In [11]:
#missing names
filterNonNullDF = flattenDF.filter(col("firstName").isNull() | col("lastName").isNull()).sort("email")
display(filterNonNullDF)

#### Summarize Data

In [13]:
from pyspark.sql.functions import countDistinct

countDistinctDF = nonNullDF.select("firstName", "lastName")\
  .groupBy("firstName")\
  .agg(countDistinct("lastName").alias("distinct_last_names"))

display(countDistinctDF)

In [14]:
#Sum the salaries
salarySumDF = nonNullDF.agg({"salary" : "sum"})
display(salarySumDF)

In [15]:
#Summary stats for salaries
nonNullDF.describe("salary").show()

#### Clean up

In [17]:
dbutils.fs.rm("/tmp/databricks-df-example.parquet", True)